In [7]:
import re
import glob

import numpy as np
import pandas as pd
import tensorflow as tf

from render import extract_params_from_path

In [27]:
def runs_to_df(paths, mode):
    data = dict(patch_size=[], filter_size=[], threshold=[], loss=[], epoch=[], step=[])
    
    for path in paths:
        params = extract_params_from_path(path)
        
        for epoch in range(31):
            step, loss = fetch_loss(path, mode, epoch)
            
            data['threshold'].append(params['threshold'])
            data['patch_size'].append(params['patch'])
            data['filter_size'].append(params['filter'])
            data['loss'].append(loss)
            data['step'].append(step)
            data['epoch'].append(epoch)
        
    return pd.DataFrame(data)
    
def fetch_loss(path, mode, epoch):
    event_files = glob.glob('{}/{}-epoch{}/events.out.*'.format(path, mode, epoch))

    loss_values = []
    step_values = []
    
    if len(event_files) == 0:
        return 0, 0
    
    for event in tf.train.summary_iterator(event_files[0]):
        for value in event.summary.value:
            if value.tag == 'loss/loss':
                loss_values.append(value.simple_value)
                step_values.append(event.step)

    return max(step_values), sum(loss_values)

In [28]:
train_df = runs_to_df(glob.glob('/tmp/mrtous/patch*'), 'train')

In [29]:
train_df['loss_per_pixel'] = train_df['loss'] / (128*train_df['step']*train_df['patch_size']**2)
train_df[train_df['epoch'] == 30].sort_values('loss')

,epoch,filter_size,loss,patch_size,step,threshold,loss_per_pixel
340,30,6,0.000000,10,0,2,NaN
464,30,3,3331.090168,5,21851,15,0.000048
371,30,3,3339.283457,5,21851,0,0.000048
402,30,3,3342.647198,5,21851,1,0.000048
495,30,3,3343.973948,5,21851,2,0.000048
433,30,3,3346.566957,5,21851,10,0.000048
526,30,3,3367.098518,5,21851,5,0.000048
154,30,3,3827.425575,10,6041,2,0.000049
30,30,3,3837.793274,10,6041,0,0.000050
185,30,3,3848.728981,10,6041,20,0.000050
